# **Data Cleaning**

## Objectives

* Clean the data by handling missing values and ensuring correct data types.
* Convert specific columns to `float` for consistency.
* Save the cleaned dataset for further analysis and modeling.

## Inputs

* outputs/datasets/collection/HousePricesRecords.csv`: The raw collected data from the Data Collection step

## Outputs

* outputs/datasets/cleaned/HousePricesCleaned.csv`: The cleaned data, ready for feature engineering and analysis

---

# Change working directory

* The notebooks are stored in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing-Issues/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing-Issues'

## Load data

Install pandas

In [5]:
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 171.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 223.3 MB/s eta 0:00:00


Load the data to check data types

In [1]:
import pandas as pd
df = pd.read_csv(f"../inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv")
df.head

<bound method NDFrame.head of       1stFlrSF  2ndFlrSF  BedroomAbvGr BsmtExposure  BsmtFinSF1 BsmtFinType1  \
0          856     854.0           3.0           No         706          GLQ   
1         1262       0.0           3.0           Gd         978          ALQ   
2          920     866.0           3.0           Mn         486          GLQ   
3          961       NaN           NaN           No         216          ALQ   
4         1145       NaN           4.0           Av         655          GLQ   
...        ...       ...           ...          ...         ...          ...   
1455       953     694.0           3.0           No           0          Unf   
1456      2073       0.0           NaN           No         790          ALQ   
1457      1188    1152.0           4.0           No         275          GLQ   
1458      1078       0.0           2.0           Mn          49          NaN   
1459      1256       0.0           3.0           No         830          BLQ   

      Bsm

Handle missing data

In [2]:
df.isnull().sum()

1stFlrSF            0
2ndFlrSF           86
BedroomAbvGr       99
BsmtExposure       38
BsmtFinSF1          0
BsmtFinType1      145
BsmtUnfSF           0
EnclosedPorch    1324
GarageArea          0
GarageFinish      235
GarageYrBlt        81
GrLivArea           0
KitchenQual         0
LotArea             0
LotFrontage       259
MasVnrArea          8
OpenPorchSF         0
OverallCond         0
OverallQual         0
TotalBsmtSF         0
WoodDeckSF       1305
YearBuilt           0
YearRemodAdd        0
SalePrice           0
dtype: int64

Remove columns with lots of missing data

In [3]:
df.drop(['EnclosedPorch', 'WoodDeckSF'], axis=1, inplace=True)


Fill missing values in numerical columns with the mean

In [6]:
df['BedroomAbvGr'].fillna(df['BedroomAbvGr'].mean(), inplace=True)
df['LotFrontage'].fillna(df['LotFrontage'].mean(), inplace=True)
df['MasVnrArea'].fillna(df['MasVnrArea'].mean(), inplace=True)
df['GarageYrBlt'].fillna(df['GarageYrBlt'].mean(), inplace=True)


Fill missing values with the most common category (mode)

In [7]:
df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0], inplace=True)
df['BsmtFinType1'].fillna(df['BsmtFinType1'].mode()[0], inplace=True)
df['GarageFinish'].fillna(df['GarageFinish'].mode()[0], inplace=True)


Check missing values in each column of the DataFrame

In [8]:
df.isnull().sum()

1stFlrSF         0
2ndFlrSF        86
BedroomAbvGr     0
BsmtExposure     0
BsmtFinSF1       0
BsmtFinType1     0
BsmtUnfSF        0
GarageArea       0
GarageFinish     0
GarageYrBlt      0
GrLivArea        0
KitchenQual      0
LotArea          0
LotFrontage      0
MasVnrArea       0
OpenPorchSF      0
OverallCond      0
OverallQual      0
TotalBsmtSF      0
YearBuilt        0
YearRemodAdd     0
SalePrice        0
dtype: int64

Check data types

In [9]:
df.dtypes

1stFlrSF          int64
2ndFlrSF        float64
BedroomAbvGr    float64
BsmtExposure     object
BsmtFinSF1        int64
BsmtFinType1     object
BsmtUnfSF         int64
GarageArea        int64
GarageFinish     object
GarageYrBlt     float64
GrLivArea         int64
KitchenQual      object
LotArea           int64
LotFrontage     float64
MasVnrArea      float64
OpenPorchSF       int64
OverallCond       int64
OverallQual       int64
TotalBsmtSF       int64
YearBuilt         int64
YearRemodAdd      int64
SalePrice         int64
dtype: object

Convert object to category 

In [11]:
df['BsmtExposure'] = df['BsmtExposure'].astype('category')
df['BsmtFinType1'] = df['BsmtFinType1'].astype('category')
df['GarageFinish'] = df['GarageFinish'].astype('category')
df['KitchenQual'] = df['KitchenQual'].astype('category')



Check data types

In [12]:
df.dtypes


1stFlrSF           int64
2ndFlrSF         float64
BedroomAbvGr     float64
BsmtExposure    category
BsmtFinSF1         int64
BsmtFinType1    category
BsmtUnfSF          int64
GarageArea         int64
GarageFinish    category
GarageYrBlt      float64
GrLivArea          int64
KitchenQual     category
LotArea            int64
LotFrontage      float64
MasVnrArea       float64
OpenPorchSF        int64
OverallCond        int64
OverallQual        int64
TotalBsmtSF        int64
YearBuilt          int64
YearRemodAdd       int64
SalePrice          int64
dtype: object

In [14]:
df.to_csv("outputs/datasets/collection/HousePricesCleaned.csv", index=False)
